# Neighborhood graphs

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [1]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

------

## Undirected grid graph

In [5]:
#% js
my $g = Graph::Grid.new(4, 6);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [6]:
my $g2 = $g.neighborhood-graph(['2_1'], d => 2)

Graph(vertexes => 11, edges => 14, directed => False)

In [7]:
#% js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## Directed grid graph

In [8]:
#% js
my $g = Graph::Grid.new(3, 4, :directed);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [9]:
my $g2 = $g.neighborhood-graph(['0_0', '1_3'], d => 1)

Graph(vertexes => 7, edges => 5, directed => True)

In [10]:
$g2.vertex-list

(0_0 0_1 0_3 1_0 1_2 1_3 2_3)

In [11]:
$g2.edges

[0_0 => 0_1 0_0 => 1_0 0_3 => 1_3 1_2 => 1_3 1_3 => 2_3]

In [12]:
#% js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## In a Petersen graph

In [61]:
#% js
my $g = Graph::Petersen.new;

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 450, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -350, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}},
        plot-label => 'Petersen graph'
    )

In [46]:
my @nns = <0 1>;
my $g2 = $g.neighborhood-graph(@nns, d => 1)

Graph(vertexes => 7, edges => 8, directed => False)

In [55]:
#%js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}},
        plot-label => "Neighborhood graph of <{@nns.join(' ')}>"
    )

In [57]:
#%html
$g.dot( 
    engine => 'neato', 
    :node-labels,
    node-font-size => 16,
    node-width => 0.2, 
    size => '3,3!', 
    highlight => [|$g2.vertex-list, |$g2.edges],
    label => "Neighborhood graph of <{@nns.join(' ')}>"
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
 Neighborhood graph of <0 1> 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 4--9 -->
 
 4--9 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 3--8 -->
 
 3--8 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 5--9 -->
 
 5--9 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 1--3 -->
 
 1--3 
 
 
<!-- 1--6 -->
 
 1--6

-----

## Neat examples

### Pick out random neighborhoods from a grid

In [16]:
my $g = Graph::Grid.new(80, 80);

my $g2 = $g.neighborhood-graph($g.vertex-list.pick(80), d => 1)

Graph(vertexes => 397, edges => 329, directed => False)

In [17]:
my $nc = $g2.weakly-connected-components.elems;
say "number of connected components: $nc";
my @comps = $g2.weakly-connected-components.pick($nc/3);

my @gs = @comps.map({ $g2.subgraph($_) });

my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
@highlight.elems

number of connected components: 75


237

In [18]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => { Salmon => @highlight[0 .. @highlight.elems div 2], Moccasin => @highlight[@highlight.elems div 2 .. *]},
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        vertex-color => 'LightSkyBlue',
        edge-color => 'RoyalBlue',
        edge-thickness => 3,
        vertex-size => 4, 
        force => {charge => {strength => -30, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

Graphviz DOT rendering:

In [19]:
#%html
$g2.dot( 
    engine => 'fdp', 
    :!node-labels, node-width => 0.2, 
    size => '12,7!', 
    highlight => { Salmon => @highlight[0 .. @highlight.elems div 2], Moccasin => @highlight[@highlight.elems div 2 .. *]}
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0_7 -->
 
 0_7 
 
 
<!-- 1_7 -->
 
 1_7 
 
 
<!-- 0_7--1_7 -->
 
 0_7--1_7 
 
 
<!-- 10_11 -->
 
 10_11 
 
 
<!-- 10_12 -->
 
 10_12 
 
 
<!-- 10_11--10_12 -->
 
 10_11--10_12 
 
 
<!-- 10_13 -->
 
 10_13 
 
 
<!-- 10_12--10_13 -->
 
 10_12--10_13 
 
 
<!-- 11_12 -->
 
 11_12 
 
 
<!-- 10_12--11_12 -->
 
 10_12--11_12 
 
 
<!-- 9_12 -->
 
 9_12 
 
 
<!-- 10_12--9_12 -->
 
 10_12--9_12 
 
 
<!-- 11_68 -->
 
 11_68 
 
 
<!-- 12_68 -->
 
 12_68 
 
 
<!-- 11_68--12_68 -->
 
 11_68--12_68 
 
 
<!-- 12_67 -->
 
 12_67 
 
 
<!-- 12_67--12_68 -->
 
 12_67--12_68 
 
 
<!-- 12_69 -->
 
 12_69 
 
 
<!-- 12_68--12_69 -->
 
 12_68--12_69 
 
 
<!-- 13_68 -->
 
 13_68 
 
 
<!-- 12_68--13_68 -->
 
 12_68--13_68 
 
 
<!-- 13_3 -->
 
 13_3 
 
 
<!-- 14_3 -->
 
 14_3 
 
 
<!-- 13_3--14_3 -->
 
 13_3--14_3 
 
 
<!-- 14_2 -->
 
 14_2 
 
 
<!-- 14_2--14_3 -->
 
 14_2--14_3 
 
 
<!-- 14_4 -->
 
 14_4 
 
 
<!-- 14_3--14_4 -->
 
 14_3--14_4 
 
 
<!-- 15_3 -->
 
 15_3 
 
 
<!-- 14_3--15_3 -->
 
 14_3--15_3 
 
 
<!-- 15_65 -->
 
 15_65 
 
 
<!-- 16_65 -->
 
 16_65 
 
 
<!-- 15_65--16_65 -->
 
 15_65--16_65 
 
 
<!-- 15_70 -->
 
 15_70 
 
 
<!-- 16_70 -->
 
 16_70 
 
 
<!-- 15_70--16_70 -->
 
 15_70--16_70 
 
 
<!-- 16_44 -->
 
 16_44 
 
 
<!-- 17_44 -->
 
 17_44 
 
 
<!-- 16_44--17_44 -->
 
 16_44--17_44 
 
 
<!-- 16_64 -->
 
 16_64 
 
 
<!-- 16_64--16_65 -->
 
 16_64--16_65 
 
 
<!-- 16_66 -->
 
 16_66 
 
 
<!-- 16_65--16_66 -->
 
 16_65--16_66 
 
 
<!-- 17_65 -->
 
 17_65 
 
 
<!-- 16_65--17_65 -->
 
 16_65--17_65 
 
 
<!-- 16_69 -->
 
 16_69 
 
 
<!-- 16_69--16_70 -->
 
 16_69--16_70 
 
 
<!-- 16_71 -->
 
 16_71 
 
 
<!-- 16_70--16_71 -->
 
 16_70--16_71 
 
 
<!-- 17_70 -->
 
 17_70 
 
 
<!-- 16_70--17_70 -->
 
 16_70--17_70 
 
 
<!-- 17_43 -->
 
 17_43 
 
 
<!-- 17_43--17_44 -->
 
 17_43--17_44 
 
 
<!-- 17_45 -->
 
 17_45 
 
 
<!-- 17_44--17_45 -->
 
 17_44--17_45 
 
 
<!-- 18_44 -->
 
 18_44 
 
 
<!-- 17_44--18_44 -->
 
 17_44--18_44 
 
 
<!-- 18_66 -->
 
 18_66 
 
 
<!-- 19_66 -->
 
 19_66 
 
 
<!-- 18_66--19_66 -->
 
 18_66--19_66 
 
 
<!-- 19_65 -->
 
 19_65 
 
 
<!-- 19_65--19_66 -->
 
 19_65--19_66 
 
 
<!-- 19_67 -->
 
 19_67 
 
 
<!-- 19_66--19_67 -->
 
 19_66--19_67 
 
 
<!-- 20_66 -->
 
 20_66 
 
 
<!-- 19_66--20_66 -->
 
 19_66--20_66 
 
 
<!-- 1_6 -->
 
 1_6 
 
 
<!-- 1_6--1_7 -->
 
 1_6--1_7 
 
 
<!-- 1_8 -->
 
 1_8 
 
 
<!-- 1_7--1_8 -->
 
 1_7--1_8 
 
 
<!-- 2_7 -->
 
 2_7 
 
 
<!-- 1_7--2_7 -->
 
 1_7--2_7 
 
 
<!-- 21_73 -->
 
 21_73 
 
 
<!-- 22_73 -->
 
 22_73 
 
 
<!-- 21_73--22_73 -->
 
 21_73--22_73 
 
 
<!-- 22_72 -->
 
 22_72 
 
 
<!-- 22_72--22_73 -->
 
 22_72--22_73 
 
 
<!-- 22_74 -->
 
 22_74 
 
 
<!-- 22_73--22_74 -->
 
 22_73--22_74 
 
 
<!-- 23_73 -->
 
 23_73 
 
 
<!-- 22_73--23_73 -->
 
 22_73--23_73 
 
 
<!-- 24_33 -->
 
 24_33 
 
 
<!-- 25_33 -->
 
 25_33 
 
 
<!-- 24_33--25_33 -->
 
 24_33--25_33 
 
 
<!-- 25_1 -->
 
 25_1 
 
 
<!-- 26_1 -->
 
 26_1 
 
 
<!-- 25_1--26_1 -->
 
 25_1--26_1 
 
 
<!-- 25_32 -->
 
 25_32 
 
 
<!-- 25_32--25_33 -->
 
 25_32--25_33 
 
 
<!-- 25_34 -->
 
 25_34 
 
 
<!-- 25_33--25_34 -->
 
 25_33--25_34 
 
 
<!-- 26_33 -->
 
 26_33 
 
 
<!-- 25_33--26_33 -->
 
 25_33--26_33 
 
 
<!-- 26_0 -->
 
 26_0 
 
 
<!-- 26_0--26_1 -->
 
 26_0--26_1 
 
 
<!-- 26_2 -->
 
 26_2 
 
 
<!-- 26_1--26_2 -->
 
 26_1--26_2 
 
 
<!-- 27_1 -->
 
 27_1 
 
 
<!-- 26_1--27_1 -->
 
 26_1--27_1 
 
 
<!-- 26_12 -->
 
 26_12 
 
 
<!-- 27_12 -->
 
 27_12 
 
 
<!-- 26_12--27_12 -->
 
 26_12--27_12 
 
 
<!-- 26_8 -->
 
 26_8 
 
 
<!-- 27_8 -->
 
 27_8 
 
 
<!-- 26_8--27_8 -->
 
 26_8--27_8 
 
 
<!-- 27_11 -->
 
 27_11 
 
 
<!-- 27_11--27_12 -->
 
 27_11--27_12 
 
 
<!-- 27_13 -->
 
 27_13 
 
 
<!-- 27_12--27_13 -->
 
 27_12--27_13 
 
 
<!-- 28_12 -->
 
 28_12 
 
 
<!-- 27_12--

In [20]:
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });
%vertex-coordinates.elems

6400

In [21]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => ($g2.vertex-list Z=> %vertex-coordinates{$g2.vertex-list}).Hash,
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        edge-thickness => 3,
        vertex-size => 0, 
    )

### Subtract random neighborhoods from a grid

In [22]:
my $g = Graph::Grid.new(30, 60);

# Not needed -- coordinates are assigned at construction.
#my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });

my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(60), d => 1).vertex-list;

my $g2 = $g.subgraph($vset.keys)

Graph(vertexes => 1739, edges => 3251, directed => False)

In [23]:
my $g3 = $g2.neighborhood-graph($g.vertex-list.pick(60), d => 2);
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;
my @gs = @comps.map({ $g3.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

24

In [25]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => $g.vertex-coordinates,
        highlight => {Ivory => @highlight},
        :$background, 
        title-color => 'none', 
        width => 1000, 
        height => 500,
        edge-thickness => 4,
        edge-color => 'Gray',
        vertex-size => 0,
        vertex-color => 'Ivory',
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )

Graphviz DOT rendering:

In [26]:
#%html
$g2.dot( 
    highlight => {Ivory => @highlight},
    :!node-labels, 
    node-width => 0,
    node-fill-color => 'Gray', 
    node-shape => 'point',
    graph-size => (10, 20),
    edge-color => 'Gray',
    edge-width => 18,
    engine => 'fdp', 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 8_56 -->
 
 8_56 
 
 
<!-- 8_57 -->
 
 8_57 
 
 
<!-- 8_56--8_57 -->
 
 8_56--8_57 
 
 
<!-- 9_56 -->
 
 9_56 
 
 
<!-- 8_56--9_56 -->
 
 8_56--9_56 
 
 
<!-- 27_57 -->
 
 27_57 
 
 
<!-- 28_57 -->
 
 28_57 
 
 
<!-- 27_57--28_57 -->
 
 27_57--28_57 
 
 
<!-- 27_58 -->
 
 27_58 
 
 
<!-- 27_57--27_58 -->
 
 27_57--27_58 
 
 
<!-- 18_0 -->
 
 18_0 
 
 
<!-- 19_0 -->
 
 19_0 
 
 
<!-- 18_0--19_0 -->
 
 18_0--19_0 
 
 
<!-- 18_1 -->
 
 18_1 
 
 
<!-- 18_0--18_1 -->
 
 18_0--18_1 
 
 
<!-- 19_6 -->
 
 19_6 
 
 
<!-- 19_7 -->
 
 19_7 
 
 
<!-- 19_6--19_7 -->
 
 19_6--19_7 
 
 
<!-- 20_6 -->
 
 20_6 
 
 
<!-- 19_6--20_6 -->
 
 19_6--20_6 
 
 
<!-- 10_36 -->
 
 10_36 
 
 
<!-- 9_36 -->
 
 9_36 
 
 
<!-- 10_36--9_36 -->
 
 10_36--9_36 
 
 
<!-- 11_36 -->
 
 11_36 
 
 
<!-- 10_36--11_36 -->
 
 10_36--11_36 
 
 
<!-- 10_37 -->
 
 10_37 
 
 
<!-- 10_36--10_37 -->
 
 10_36--10_37 
 
 
<!-- 14_5 -->
 
 14_5 
 
 
<!-- 15_5 -->
 
 15_5 
 
 
<!-- 14_5--15_5 -->
 
 14_5--15_5 
 
 
<!-- 14_6 -->
 
 14_6 
 
 
<!-- 14_5--14_6 -->
 
 14_5--14_6 
 
 
<!-- 13_20 -->
 
 13_20 
 
 
<!-- 14_20 -->
 
 14_20 
 
 
<!-- 13_20--14_20 -->
 
 13_20--14_20 
 
 
<!-- 13_21 -->
 
 13_21 
 
 
<!-- 13_20--13_21 -->
 
 13_20--13_21 
 
 
<!-- 11_38 -->
 
 11_38 
 
 
<!-- 11_39 -->
 
 11_39 
 
 
<!-- 11_38--11_39 -->
 
 11_38--11_39 
 
 
<!-- 12_38 -->
 
 12_38 
 
 
<!-- 11_38--12_38 -->
 
 11_38--12_38 
 
 
<!-- 13_0 -->
 
 13_0 
 
 
<!-- 14_0 -->
 
 14_0 
 
 
<!-- 13_0--14_0 -->
 
 13_0--14_0 
 
 
<!-- 13_1 -->
 
 13_1 
 
 
<!-- 13_0--13_1 -->
 
 13_0--13_1 
 
 
<!-- 12_6 -->
 
 12_6 
 
 
<!-- 12_7 -->
 
 12_7 
 
 
<!-- 12_6--12_7 -->
 
 12_6--12_7 
 
 
<!-- 13_6 -->
 
 13_6 
 
 
<!-- 12_6--13_6 -->
 
 12_6--13_6 
 
 
<!-- 7_25 -->
 
 7_25 
 
 
<!-- 8_25 -->
 
 8_25 
 
 
<!-- 7_25--8_25 -->
 
 7_25--8_25 
 
 
<!-- 7_26 -->
 
 7_26 
 
 
<!-- 7_25--7_26 -->
 
 7_25--7_26 
 
 
<!-- 2_19 -->
 
 2_19 
 
 
<!-- 2_20 -->
 
 2_20 
 
 
<!-- 2_19--2_20 -->
 
 2_19--2_20 
 
 
<!-- 3_19 -->
 
 3_19 
 
 
<!-- 2_19--3_19 -->
 
 2_19--3_19 
 
 
<!-- 9_20 -->
 
 9_20 
 
 
<!-- 9_21 -->
 
 9_21 
 
 
<!-- 9_20--9_21 -->
 
 9_20--9_21 
 
 
<!-- 7_58 -->
 
 7_58 
 
 
<!-- 8_58 -->
 
 8_58 
 
 
<!-- 7_58--8_58 -->
 
 7_58--8_58 
 
 
<!-- 7_59 -->
 
 7_59 
 
 
<!-- 7_58--7_59 -->
 
 7_58--7_59 
 
 
<!-- 11_46 -->
 
 11_46 
 
 
<!-- 12_46 -->
 
 12_46 
 
 
<!-- 11_46--12_46 -->
 
 11_46--12_46 
 
 
<!-- 11_47 -->
 
 11_47 
 
 
<!-- 11_46--11_47 -->
 
 11_46--11_47 
 
 
<!-- 23_58 -->
 
 23_58 
 
 
<!-- 24_58 -->
 
 24_58 
 
 
<!-- 23_58--24_58 -->
 
 23_58--24_58 
 
 
<!-- 23_59 -->
 
 23_59 
 
 
<!-- 23_58--23_59 -->
 
 23_58--23_59 
 
 
<!-- 27_9 -->
 
 27_9 
 
 
<!-- 28_9 -->
 
 28_9 
 
 
<!-- 27_9--28_9 -->
 
 27_9--28_9 
 
 
<!-- 5_50 -->
 
 5_50 
 
 
<!-- 6_50 -->
 
 6_50 
 
 
<!-- 5_50--6_50 -->
 
 5_50--6_50 
 
 
<!-- 5_51 -->
 
 5_51 
 
 
<!-- 5_50--5_51 -->
 
 5_50--5_51 
 
 
<!-- 21_2 -->
 
 21_2 
 
 
<!-- 21_3 -->
 
 21_3 
 
 
<!-- 21_2--21_3 -->
 
 21_2--21_3 
 
 
<!-- 22_2 -->
 
 22_2 
 
 
<!-- 21_2--22_2 -->
 
 21_2--22_2 
 
 
<!-- 4_59 -->
 
 4_59 
 
 
<!-- 5_59 -->
 
 5_59 
 
 
<!-- 4_59--5_59 -->
 
 4_59--5_59 
 
 
<!-- 28_28 -->
 
 28_28 
 
 
<!-- 28_29 -->
 
 28_29 
 
 
<!-- 28_28--28_29 -->
 
 28_28--28_29 
 
 
<!-- 29_28 -->
 
 29_28 
 
 
<!-- 28_28--29_28 -->
 
 28_28--29_28 
 
 
<!-- 24_3 -->
 
 24_3 
 
 
<!-- 25_3 -->
 
 25_3 
 
 
<!-- 24_3--25_3 -->
 
 24_3--25_3 
 
 
<!-- 24_4 -->
 
 24_4 
 
 
<!-- 24_3--24_4 -->
 
 24_3--24_4 
 
 
<!-- 6_38 -->
 
 6_38 
 
 
<!-- 6_39 -->
 
 6_39 
 
 
<!-- 6_38--6_39 -->
 
 6_38--6_39 
 
 
<!-- 7_38 -->
 
 7_38 
 
 
<!-- 6_38--7_38 -->
 
 6_38--7_38 
 
 
<!-- 28_15 -->
 
 28_15 
 
 
<!-- 29_15 -->
 
 29_15 
 
 
<!-- 28_15--29_15 -->
 
 28_15--29_15 
 
 
<!-- 28_16 -->
 
 28_16 
 
 
<!-